# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import time

In [2]:
URL_PREFIX = "https://books.toscrape.com/catalogue/"

CATALOGUE_NUMBER_OF_PAGES = 51

rating_map = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5
}



def get_book_urls(min_rating, max_price):
    """
    Returns:
        urls[str]: A list of URLs (strings) to the detail pages of books matching the criteria.
    """

    urls = []

    # iterate through all pages in the catalogue
    for n in range(1, CATALOGUE_NUMBER_OF_PAGES):

        catalogue_page = f"{URL_PREFIX}/page-{n}.html"
        print(f"Getting books from page... {n}")

        try:
            res = requests.get(catalogue_page)
            soup = BeautifulSoup(res.content)

            products = soup.find_all("article", attrs={"class": "product_pod"})
            
            # iterate through all products in each page 
            for product in products:

                # get rating for the current product
                rating = None
                rating_classes = product.find(attrs={"class": "star-rating"}).get("class", [])
                for class_name in rating_classes:
                    if class_name in rating_map:
                        rating = rating_map[class_name]
                
                # get price for the current product
                price_tag = product.find(attrs={"class": "price_color"})
                price = price_tag.text.strip().lstrip('£')
                price = float(price)

                # if book matches the criteria, get url and add it to the list of urls
                if rating >= min_rating and price <= max_price:
                    href = product.find("a").get("href")
                    product_url = URL_PREFIX + href
                    urls.append(product_url)

            time.sleep(0.1)
        except Exception as e:
            print(f"\n\nError getting page... {n}")
            print(catalogue_page)
            print(e)

    return urls



def scrape_books(min_rating=5.0, max_price=999):
    """
    Scrape book details from Books to Scrape website, filtering by minimum rating and maximum price.

    Parameters:
        min_rating (float): Minimum book rating (e.g. 3.0, 4.0, etc.).
        max_price (float): Maximum price allowed for books.

    Returns:
        List[dict]: A list of dictionaries, each containing:
            - url (str): URL of the book detail page.
            - upc (str): Book UPC code.
            - title (str): Book title.
            - price (float): Price of the book (without currency symbol).
            - rating (int): Numeric rating (1 to 5).
            - genre (str): Book genre.
            - availability (int): Number of books in stock.
            - description (str): Book description text.
    """
    data = []

    print("Getting book urls...")
    urls = get_book_urls(min_rating, max_price)
    print(f"Number of urls collected and meeting that criteria: {len(urls)}")

    print("\n")
    print(f"Scraping data for those {len(urls)} books...")
    
    total = len(urls)
    checkpoints = {
        int(total * 0.2),
        int(total * 0.4),
        int(total * 0.6),
        int(total * 0.8),
    }

    for i, url in enumerate(urls, start=1):

        # display progress...
        if i in checkpoints:
            percent = int((i / total) * 100)
            print(f"Progress: {percent}% ({i} of {total})")

        # send request
        res = requests.get(url)
        soup = BeautifulSoup(res.content)
        time.sleep(0.1)

        # URL
        # print(url)

        # UPC
        upc_th = soup.find("th", string="UPC")
        upc = upc_th.find_next_sibling("td").text.strip()

        # Title
        title = soup.select_one(".product_main h1").text

        # Price
        price = soup.select_one(".price_color").text.strip().lstrip('£')
        price = float(price)

        # Rating
        rating = None
        rating_classes = soup.find(attrs={"class": "star-rating"}).get("class", [])
        for class_name in rating_classes:
            if class_name in rating_map:
                rating = rating_map[class_name]

        # Genre
        breadcrumb = soup.select("ul.breadcrumb li a")
        genre = breadcrumb[-1].text.strip()

        # Availability / Stock
        availability_tag = soup.select_one("p.instock.availability")
        availability_text = availability_tag.get_text(strip=True)
        match = re.search(r'(\d+)', availability_text)
        number_available = int(match.group(1)) if match else 0

        # Description
        desc_header = soup.find(id="product_description")
        if desc_header:
            desc_p = desc_header.find_next_sibling("p")
            description = desc_p.get_text(strip=True) if desc_p else ""
        else:
            description = ""

        # Save all details into a dictionary and append to data[]
        book_data = {
            "upc": upc,
            "title": title,
            "price": float(price),
            "rating": rating,
            "genre": genre,
            "availability": number_available,
            "description": description,
            "url": url,
        }

        data.append(book_data)

    print("Done! :) ")
    return data



In [3]:

books = scrape_books(min_rating=4.0, max_price=20)


Getting book urls...
Getting books from page... 1
Getting books from page... 2
Getting books from page... 3
Getting books from page... 4
Getting books from page... 5
Getting books from page... 6
Getting books from page... 7
Getting books from page... 8
Getting books from page... 9
Getting books from page... 10
Getting books from page... 11
Getting books from page... 12
Getting books from page... 13
Getting books from page... 14
Getting books from page... 15
Getting books from page... 16
Getting books from page... 17
Getting books from page... 18
Getting books from page... 19
Getting books from page... 20
Getting books from page... 21
Getting books from page... 22
Getting books from page... 23
Getting books from page... 24
Getting books from page... 25
Getting books from page... 26
Getting books from page... 27
Getting books from page... 28
Getting books from page... 29
Getting books from page... 30
Getting books from page... 31
Getting books from page... 32
Getting books from page... 3

In [4]:
import pandas as pd

books_df = pd.DataFrame(books)

books_df.shape


(75, 8)

In [5]:
books_df.head()

,upc,title,price,rating,genre,availability,description,url
0,ce6396b0f23f6ecc,Set Me Free,17.46,5,Young Adult,19,Aaron Ledbetter’s future had been planned out ...,https://books.toscrape.com/catalogue/set-me-fr...
1,6258a1f6a6dcfe50,The Four Agreements: A Practical Guide to Pers...,17.66,5,Spirituality,18,"In The Four Agreements, don Miguel Ruiz reveal...",https://books.toscrape.com/catalogue/the-four-...
2,6be3beb0793a53e7,Sophie's World,15.94,5,Philosophy,18,A page-turning novel that is also an explorati...,https://books.toscrape.com/catalogue/sophies-w...
3,657fe5ead67a7767,Untitled Collection: Sabbath Poems 2014,14.27,4,Poetry,16,"More than thirty-five years ago, when the weat...",https://books.toscrape.com/catalogue/untitled-...
4,51653ef291ab7ddc,This One Summer,19.49,4,Sequential Art,16,"Every summer, Rose goes with her mom and dad t...",https://books.toscrape.com/catalogue/this-one-...
